In [1]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import layers, models

In [2]:
# VGG architecture
def VGGNet(input_shape, num_classes):
    model = models.Sequential()

    # Block 1
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 2
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 3
    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 4
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 5
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

    # Flatten
    model.add(layers.Flatten())

    # Dense layers
    model.add(layers.Dense(4096, activation='relu'))
    model.add(layers.Dense(4096, activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))  # Output layer with softmax activation for multi-class classification

    return model


In [3]:
train_dir = '/Users/sandundesilva/Documents/4th year/machine vision/cw final/backend/dataset/train'
test_dir = '/Users/sandundesilva/Documents/4th year/machine vision/cw final/backend/dataset/test'

In [4]:

# Define constants
input_shape = (224, 224, 3) 
num_classes = 3
batch_size = 32
epochs = 10


In [5]:
# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical')

Found 3920 images belonging to 3 classes.
Found 341 images belonging to 3 classes.


Python(5753) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [7]:
# Define the model
model = VGGNet(input_shape, num_classes)

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 64)      0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 56, 56, 128)       0         
 g2D)                                                   

In [9]:
# Compile the model
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [10]:
# Define callbacks
checkpoint = ModelCheckpoint('vggnet_model.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

In [11]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size,
    callbacks=[checkpoint])

Epoch 1/10
122/122 [==============================] - ETA: 0s - loss: 1.2809 - accuracy: 0.3318 
Epoch 1: val_accuracy improved from -inf to 0.32812, saving model to vggnet_model.h5


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


122/122 [==============================] - 1508s 12s/step - loss: 1.2809 - accuracy: 0.3318 - val_loss: 1.0987 - val_accuracy: 0.3281
Epoch 2/10
122/122 [==============================] - ETA: 0s - loss: 1.0987 - accuracy: 0.3326 
Epoch 2: val_accuracy improved from 0.32812 to 0.34688, saving model to vggnet_model.h5
122/122 [==============================] - 1487s 12s/step - loss: 1.0987 - accuracy: 0.3326 - val_loss: 1.0986 - val_accuracy: 0.3469
Epoch 3/10
122/122 [==============================] - ETA: 0s - loss: 1.0988 - accuracy: 0.3364 
Epoch 3: val_accuracy did not improve from 0.34688
122/122 [==============================] - 1463s 12s/step - loss: 1.0988 - accuracy: 0.3364 - val_loss: 1.0988 - val_accuracy: 0.3281
Epoch 4/10
122/122 [==============================] - ETA: 0s - loss: 1.0988 - accuracy: 0.3246 
Epoch 4: val_accuracy did not improve from 0.34688
122/122 [==============================] - 1468s 12s/step - loss: 1.0988 - accuracy: 0.3246 - val_loss: 1.0988 - val_

In [12]:
# Evaluate the model
loss, accuracy = model.evaluate(test_generator)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')

11/11 [==============================] - 35s 3s/step - loss: 1.0986 - accuracy: 0.3343
Test loss: 1.0986
Test accuracy: 0.3343
